In [1]:
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd
from datetime import datetime
import pyodbc


In [2]:
SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

creds = Credentials.from_service_account_file(
    "service_account.json",
    scopes=SCOPES
)

client = gspread.authorize(creds)
sheet_id = "1x911fOL9VzTMutURHe4m2uO-DU0UU1JT0TgJITM1SJo"
sheet = client.open_by_key(sheet_id).sheet1
records = sheet.get_all_records()
shine_df = pd.DataFrame(records)
shine_df.head()


,Timestamp,Title,"Author ( Last name, First Name)",Publication Year,Publication Type,Source,DOI/URL,Primary Research Focus,Keywords / Tags,Annotated Notes,Methodology,Relevance to Distance Education
0,13/12/2025 12:43:04,Teaching Writing Online: Pedagogical Strategie...,"Smith, J.; Alvarez, M.",2021,Journal Article,Computers and Composition,https://doi.org/10.1016/j.compcom.2021.102654,"Online Writing Instruction, Accessibility","online writing, accessibility, universal desig...",The article highlights the importance of Unive...,Qualitative,5
1,13/12/2025 12:48:05,Faculty Preparation for Online Teaching: Chall...,"Williams, R.; Patel, S.",2020,Conference Paper,Proceedings of the Distance Teaching & Learnin...,N/A,"Faculty Training, Technology Tools","faculty development, online teaching, instruct...",The authors identify gaps in faculty preparedn...,Mixed Methods,5
2,13/12/2025 12:51:38,Assessment and Feedback Practices in Online Hi...,"Garcia, P.; Thompson, E.",2018,Journal Article,The Internet and Higher Education,https://doi.org/10.1016/j.iheduc.2018.05.003,"Distance Education Pedagogy, Assessment & Feed...","online assessment, feedback, grading practices...","The research shows that frequent, formative fe...",Quantitative,4
3,13/12/2025 12:53:02,Accessibility Compliance and Universal Design ...,"Brown, T.; Nguyen, H.",2022,Report,National Center for Accessible Education,https://www.ncae.org/reports/ude-distance-ed,"Distance Education Pedagogy, Accessibility","accessibility compliance, universal design, on...",The report outlines common accessibility viola...,Theoretical / Conceptual,5


In [3]:
shine_df.columns = shine_df.columns.str.strip()

shine_df = shine_df.rename(columns={
    "Timestamp": "source_timestamp",
    "Title": "title",
    "Author ( Last name, First Name)": "authors",
    "Publication Year": "publication_year",
    "Publication Type": "publication_type",
    "Source": "source",
    "DOI/URL": "doi_url",
    "Primary Research Focus": "primary_research_focus",
    "Keywords / Tags": "keywords",
    "Annotated Notes": "annotation",
    "Methodology": "methodology",
    "Relevance to Distance Education": "relevance_score"
})


In [4]:
def parse_google_timestamp(ts):
    if not ts:
        return None

    for fmt in ("%m/%d/%Y %H:%M:%S", "%d/%m/%Y %H:%M:%S"):
        try:
            return datetime.strptime(ts, fmt)
        except ValueError:
            continue

    raise ValueError(f"Unrecognized timestamp format: {ts}")


In [5]:
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost;"
    "DATABASE=SHINE_DB;"
    "UID=shine_user;"
    "PWD=Qweasdzxc@2811"
)

cursor = conn.cursor()


In [6]:
insert_sql = """
INSERT INTO shine_raw.research_entries_raw (
    title,
    authors,
    publication_year,
    publication_type,
    source,
    keywords,
    annotation,
    methodology,
    relevance_score,
    source_timestamp
)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

for _, row in shine_df.iterrows():

    source_ts = parse_google_timestamp(row.get("source_timestamp"))

    cursor.execute(
        insert_sql,
        row.get("title"),
        row.get("authors"),
        str(row.get("publication_year")),
        row.get("publication_type"),
        row.get("source"),
        row.get("keywords"),
        row.get("annotation"),
        row.get("methodology"),
        str(row.get("relevance_score")),
        source_ts
    )

print("Data successfully loaded into shine_raw.research_entries_raw")


Data successfully loaded into shine_raw.research_entries_raw


In [7]:
def execute_sql_file(cursor, path):
    with open(path, "r", encoding="utf-8") as f:
        cursor.execute(f.read())

sql_files = [
    "shine_transformed_research_entries.sql",
    "shine_transformed_authors.sql",
    "shine_transformed_entry_authors.sql",
    "shine_transformed.keywords.sql",
    "shine_transformed_entry_keywords.sql"
]

try:
    for file in sql_files:
        execute_sql_file(cursor, file)
    conn.commit()
    print("Raw → Transformed load completed successfully")
except Exception as e:
    conn.rollback()
    raise e
    print("Raw → Transformed load failed and performed rollback")
finally:
    cursor.close()
    conn.close()




Raw → Transformed load completed successfully


In [8]:
import sqlite3
print(sqlite3.sqlite_version)

3.50.2


In [10]:
import pyodbc
import sqlite3
import pandas as pd

# --- SQL Server connection ---
sqlserver_conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost;"
    "DATABASE=SHINE_DB;"
    "UID=shine_user;"
    "PWD=Qweasdzxc@2811"
)

# --- Read from view ---
query = """
SELECT *
FROM shine_transformed.vw_research_search
"""
df = pd.read_sql(query, sqlserver_conn)

# --- SQLite connection (creates file) ---
sqlite_conn = sqlite3.connect("shine.db")

# --- Write to SQLite ---
df.to_sql(
    "research_search",
    sqlite_conn,
    if_exists="replace",
    index=False
)

sqlite_conn.close()
sqlserver_conn.close()

print("SQLite database 'shine.db' created successfully")


SQLite database 'shine.db' created successfully


C:\Users\gobbu\AppData\Local\Temp\ipykernel_25996\2258861506.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, sqlserver_conn)


In [12]:
import pyodbc
import sqlite3
import pandas as pd

sqlserver_conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost;"
    "DATABASE=SHINE_DB;"
    "UID=shine_user;"
    "PWD=Qweasdzxc@2811"
)

df = pd.read_sql(
    "SELECT * FROM shine_transformed.vw_research_search",
    sqlserver_conn
)

sqlite_conn = sqlite3.connect("shine.db")

df.to_sql("research_search", sqlite_conn, if_exists="replace", index=False)

sqlite_conn.close()
sqlserver_conn.close()

print("shine.db created")


shine.db created


C:\Users\gobbu\AppData\Local\Temp\ipykernel_25996\3130134001.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


In [13]:
import sqlite3

conn = sqlite3.connect("shine.db")
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

conn.close()


[('research_search',)]


In [15]:
import pyodbc
import sqlite3
import pandas as pd

# -----------------------------
# SQL Server connection
# -----------------------------
sqlserver_conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost;"
    "DATABASE=SHINE_DB;"
    "UID=shine_user;"
    "PWD=Qweasdzxc@2811"
)

# -----------------------------
# Pull data from VIEW
# -----------------------------
query = "SELECT * FROM shine_transformed.vw_research_search"
df = pd.read_sql(query, sqlserver_conn)

print("Rows fetched from SQL Server:", len(df))
print("Columns:", df.columns.tolist())

# 🚨 IMPORTANT: ensure data exists
if df.empty:
    raise Exception("No data fetched from SQL Server view")

# -----------------------------
# Create SQLite DB
# -----------------------------
sqlite_conn = sqlite3.connect("shine.db")

df.to_sql(
    "research_search",   # table name
    sqlite_conn,
    if_exists="replace",
    index=False
)

# -----------------------------
# VERIFY SQLite content
# -----------------------------
tables = pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table';",
    sqlite_conn
)

print("SQLite tables:", tables)

sqlite_conn.close()
sqlserver_conn.close()

print("shine.db recreated successfully")


Rows fetched from SQL Server: 4
Columns: ['entry_id', 'title', 'publication_year', 'publication_type', 'source', 'annotation', 'methodology', 'relevance_score', 'source_timestamp', 'created_at', 'authors', 'keywords']
SQLite tables:               name
0  research_search
shine.db recreated successfully


C:\Users\gobbu\AppData\Local\Temp\ipykernel_25996\2063507115.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, sqlserver_conn)
